# Self-Reflecting Agents with Guardrails on Amazon Bedrock

> **As featured in AWS re:Invent 2024 session AIM325**

This notebook demonstrates the pattern discussed in the AWS ML Blog post *["Reducing hallucinations in large language models with custom intervention using Amazon Bedrock Agents"](https://aws.amazon.com/blogs/machine-learning/reducing-hallucinations-in-large-language-models-with-custom-intervention-using-amazon-bedrock-agents/)*: We'll set up an [Amazon Bedrock Agent](https://aws.amazon.com/bedrock/agents/) to conduct an additional check for hallucinations when it deems necessary - and then either 1/ continue responding to the user, or 2/ request them to wait while a human operator is notified to join and assist.


## Overview

As shown in the [bedrock-knowledge-base-guardrails](../bedrock-knowledge-base-guardrails/) example, [Amazon Bedrock Knowledge Bases](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html) provide a fully-managed Retrieval-Augmented Generation (RAG) solution, supporting built-in [contextual grounding and answer relevance checks](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-contextual-grounding-check.html) by Amazon Bedrock Guardrails.

In this example though, we'll explore implementing **custom** grounding checks using Open Source library [Ragas](https://docs.ragas.io/en/latest/); and **agentic** orchestration with [Amazon Bedrock Agents](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-how.html).

The implemented flow will be as follows, shown also in the architecture diagram below:

0. (Offline / in advance) Trusted documents are ingested from Amazon S3 into the Amazon Bedrock Knowledge Base
1. The user asks the agent a question relevant to the documentation
2. The **agent decides** that the knowledge base will be useful to support its answer, and searches the Bedrock Knowledge Base for relevant content
3. The Bedrock Knowledge Base vectorizes the search query and performs the search in Amazon OpenSearch Serverless - the underlying vector store
4. Relevant documentation "chunks" are retrieved
5. Based on the retrieved chunks, an answer is drafted by the LLM
6. The **agent decides** (based on its system prompt guidance) that this is an appropriate situation to invoke its configured hallucination/grounding checker tool
7. The agent tool request is mapped to an AWS Lambda function which checks the proposed answer using Ragas metrics
8. If the check fails, the tool sends an Amazon SNS notification requesting a human operator to intervene and support
9. The checker tool responds telling the agent how to respond based on the result of the check
10. The agent either returns the validated response to the user, or an override message asking them to wait while a human operator joins the chat.

![](images/lab2-reinvent-arch-diagram-v1.png "This image shows the retrieval augmented generation (RAG) system design setup with knowledge bases, S3, and AOSS. Knowledge corpus is ingested into a vector database using Amazon Bedrock Knowledge Base Agent and then RAG approach is used to work question answering. The question is converted into embeddings followed by semantic similarity search to get similar documents. With the user prompt being augmented with the RAG search response, the LLM is invoked to get the final raw response for the user.")

## Environment setup and configuration

**Permissions:**
- *This notebook* (i.e. your AWS CLI credentials, or your SageMaker Execution Role if running in SageMaker AI) needs access to invoke models, query Knowledge Bases, and create+invoke Agents in Amazon Bedrock.
- *You* (i.e. your AWS Console user) may need access to deploy an OpenSearch Serverless-backed Bedrock Knowledge Base, unless this has been done already for you (in an AWS-led event or having run the [bedrock-knowledge-base-guardrails sample](../bedrock-knowledge-base-guardrails/) already)

**Kernel and libraries:**

This notebook uses the same libraries as defined in the top-level [pyproject.toml](../../pyproject.toml) in this sample repository. Run the cells below to install those and then restart your notebook kernel, if you don't have them already:

In [ ]:
%pip install -e ../..

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>🔄 Restart the kernel after installing</h4>
    <p>
        If you run the above cell you'll need to restart the notebook kernel afterwards, for the
        installations to take full effect.
    </p>
    <p>
        Note that you may see some error notices about dependency conflicts in SageMaker Studio
        environments, but this is okay as long as the installations are completed.
    </p>
</div>
<br/>

With the relevant libraries installed, we're ready to load them up and connect to the AWS services that'll be used in the rest of the notebook:

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json
import logging
import pprint
import time
import uuid
import warnings

# External Dependencies:
import boto3  # AWS SDK for Python
from botocore.config import Config as BotoConfig
import pandas as pd

# Local Helper Utilities:
from hallucination_utils import bedrock_agents as bedrock_agent_utils
# from agent_utilities.agents_utils import *
# from agent_utilities.agents_infra_utils_one_kb_setup import *

# Connect to AWS Services:
boto_session = boto3.Session()  # Can optionally override 'region_name' here if wanted
bedrock_config = BotoConfig(
    # Override default retry & timeout config for (maybe long-running/throttled) FM invocations
    retries={"max_attempts": 3, "mode": "adaptive"},
    read_timeout=1000,
    connect_timeout=1000,
)
bedrock_runtime = boto_session.client("bedrock-runtime", config=bedrock_config)
bedrock_agent_client = boto_session.client("bedrock-agent")
bedrock_agent_runtime = boto_session.client(
    "bedrock-agent-runtime", config=bedrock_config
)
lambda_client = boto3.client("lambda")
sts_client = boto3.client("sts")

# Setup for logging/printing outputs:
logging.basicConfig(
    format="%(asctime)s {%(filename)s:%(lineno)d} %(levelname)s %(message)s",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore")
pp = pprint.PrettyPrinter(width=41, compact=True)

This notebook and agent has been tested with Anthropic Claude 3 Sonnet for text generation, as configured below.

It should be possible to run with other models instead if needed (for e.g. due to regional availability or updates over time), but this may impact the observed behaviour and metrics:

In [ ]:
llm_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

## Test Bedrock inference

In [ ]:
input_prompt = "Who was the first person to land on the sun?"

response = bedrock_runtime.converse(
    modelId=llm_model_id,
    messages=[{"role": "user", "content": [{"text": input_prompt}]}],
    inferenceConfig={
        "maxTokens": 500,
        "temperature": 1.0,
        "topP": 0.999,
    },
)
for c in response["output"]["message"]["content"]:
    print(c["text"])

## The target dataset and use-case

To demonstrate the pattern, we'll use (an [outdated version](https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/1fa309f2-c771-42d5-87bc-e8f919e7bcc9/bedrock-ug.pdf) of) the publicly available [Amazon Bedrock User Guide](https://docs.aws.amazon.com/pdfs/bedrock/latest/userguide/bedrock-ug.pdf) as an example reference document to form the "knowledge base".

We've created a small set of test questions and reference "ground truth" answers in [test-questions.csv](test-questions.csv) to help quantify performance:

In [ ]:
questions_df = pd.read_csv("./test-questions.csv", sep=",")

with pd.option_context("display.max_colwidth", None):
    display(questions_df)

## Set up the Knowledge Base

If you already completed the [bedrock-knowledge-base-guardrails](../bedrock-knowledge-base-guardrails/) sample, you may already have deployed the sample Bedrock Knowledge Base and can proceed straight to the next code cell to look up its ID.

▶️ **Otherwise, you'll need to deploy** this sample knowledge base first. We provide an [AWS CloudFormation](https://aws.amazon.com/cloudformation/resources/templates/) template to make this setup as simple as possible, in [/infra/Bedrock-Knowledge-Base.yaml](../../infra/Bedrock-Knowledge-Base.yaml). You can upload this template yourself through the [CloudFormation Console](https://console.aws.amazon.com/cloudformation/home?#/stacks/create) - or click the button below to get started with a version we've already published to Amazon S3:

[![Launch Stack](https://s3.amazonaws.com/cloudformation-examples/cloudformation-launch-stack.png)](https://console.aws.amazon.com/cloudformation/home?#/stacks/create/review?templateURL=https://s3.amazonaws.com/ws-assets-prod-iad-r-iad-ed304a55c2ca1aee/1fa309f2-c771-42d5-87bc-e8f919e7bcc9/Bedrock-Knowledge-Base.yaml&stackName=HallucinationKBDemo "Launch Stack")

<div style="border: 4px solid coral; text-align: left; padding: 15px;">
    <strong>⏰ This deployment can take ~10-15 minutes to complete</strong>
</div>
<br/>

You can check in the [Bedrock Knowledge Bases Console](https://console.aws.amazon.com/bedrock/home?#/knowledge-bases) that your knowledge base is deployed successfully, and run the cell below to look up its ID:

In [ ]:
kb_name = "bedrock-userguide-demo"

kb_id = None
kb_list = bedrock_agent_client.list_knowledge_bases()["knowledgeBaseSummaries"]
for kb in kb_list:
    if kb["name"] == kb_name:
        kb_id = kb["knowledgeBaseId"]

if kb_id is None:
    raise ValueError(
        "Couldn't find pre-created Bedrock Knowledge Base. Please follow the instructions "
        "above to deploy the sample knowledge base, double-check its name matches '%s' configured "
        "above, then re-run this cell." % kb_name
    )
print(f"Using existing Bedrock Knowledge Base with ID: {kb_id}")

### Test querying the Knowledge Base

Bedrock Knowledge Bases support either retrieving plain search results (via the [Retrieve API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_Retrieve.html)), or conducting an end-to-end retrieval-augmented answer generation (via [RetrieveAndGenerate](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html) or [RetrieveAndGenerateStream](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerateStream.html)).

Before connecting your Knowledge Base to an Agent, let's check it's working by running a quick query:

In [ ]:
response = bedrock_agent_runtime.retrieve(
    knowledgeBaseId=kb_id,
    retrievalQuery={"text": "How do Bedrock Guardrails work?"},
)
search_results = response["retrievalResults"]
print(f"Got {len(search_results)} search results")
for ix, r in enumerate(search_results):
    print("==========")
    print(f"Result {ix + 1}")
    s3_loc = r.get("location", {}).get("s3Location", {}).get("uri")
    if s3_loc:
        print(f"--[from: {s3_loc}]")
    print(r["content"].get("text"))

## Set up the hallucination detection tool

Next, we need to deploy:

1. The AWS Lambda Function implementing the custom hallucination checker tool, and
2. An IAM Execution Role for Amazon Bedrock Agents, with sufficient permissions to invoke both this Lambda function and the Bedrock Knowledge Base from earlier.

We've implemented this in another deployable CloudFormation template - in [infra/Hallucination-Notifier-Function.yaml](infra/Hallucination-Notifier-Function.yaml). You can upload this template yourself through the [CloudFormation Console](https://console.aws.amazon.com/cloudformation/home?#/stacks/create) - or click the button below to get started with a version we've already published to Amazon S3:

[![Launch Stack](https://s3.amazonaws.com/cloudformation-examples/cloudformation-launch-stack.png)](https://console.aws.amazon.com/cloudformation/home?#/stacks/create/review?templateURL=https://s3.amazonaws.com/ws-assets-prod-iad-r-iad-ed304a55c2ca1aee/1fa309f2-c771-42d5-87bc-e8f919e7bcc9/bedrock-agent-self-reflection/Hallucination-Notifier-Function.yaml&stackName=HalluNotifierTool "Launch Stack")

Once the deployment is completed, check the stack's **Outputs** (from SAM CLI or the [CloudFormation Console](https://console.aws.amazon.com/cloudformation/home)) to fill in your Agent execution role and Lambda function ARNs below:

In [ ]:
agent_role_arn = ""  # TODO from CloudFormation stack outputs
lambda_function_arn = ""  # TODO from CloudFormation stack outputs

if not agent_role_arn:
    raise ValueError(
        "Please set the agent_role_arn variable to the ARN of the Agent execution role you deployed via CloudFormation."
    )
if not lambda_function_arn:
    raise ValueError(
        "Please set the lambda_function_arn variable to the ARN of the Lambda function you deployed via CloudFormation."
    )

### (Optionally) Subscribe to the topic for email notifications

If you'd like, you can subscribe to the created SNS topic to receive the agent's human support request notifications in your email:

1. Navigate to ['Create subscription' in the Amazon SNS Console](https://console.aws.amazon.com/sns/v3/home?#/create-subscription)
2. For **Topic ARN**, refer to the `OutboundSnsTopicArn` output of your CloudFormation Stack as used above
3. For **Protocol**, select `Email`
4. For **Endpoint**, enter your email address
5. Leave other/advanced settings as default, and finish creating the subscription

If successful, you should receive an email straight away confirming your subscription has been set up.

This step is not mandatory though: Even if you don't receive the email notifications, you'll be able to see from the Agent's response whether the hallucination check passed or failed and requested human intervention.

## Create the Amazon Bedrock Agent

Once the supported infrastructure is in place, we can use the AWS Python SDK (`boto3`) to create and configure your Amazon Bedrock Agent from ehre in the notebook.

First, we'll create the agent itself specifying basic configurations including its IAM role and system prompt(s):

In [ ]:
create_agent_resp = bedrock_agent_client.create_agent(
    agentName="self-reflection-demo-agent",
    agentResourceRoleArn=agent_role_arn,
    description="Amazon Bedrock expert with self-reflection to escalate to humans where needed",
    foundationModel=llm_model_id,
    idleSessionTTLInSeconds=3600,
    instruction="""You are a question answering agent that helps customers answer questions from 
the Amazon Bedrock User Guide inside the associated knowledge base. If you have a hallucination
detection tool available, you *must* use it to check any responses where the knowledge base was
searched, before returning the response to the user.
""",
)

agent_id = create_agent_resp["agent"]["agentId"]
print(f"Created Agent with ID: {agent_id}")

You'll also be able to see your created agents in the [Agents section of the Amazon Bedrock Console](https://console.aws.amazon.com/bedrock/home?#/agents), including the automatically-assigned unique ID visible on your agent's detail page.

## Associate the Knowledge Base

With the agent created, we can now attach it to the sample knowledge base:

In [ ]:
assoc_kb_resp = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion="DRAFT",
    description=f"Use the {kb_id} Amazon Bedrock documentation Knowledge Base to answer questions",
    knowledgeBaseId=kb_id,
)

### (Optional) Test the agent with Knowledge Base only

Before we go ahead and attach the hallucination checker tool, you can optionally deploy and test the Agent with only the knowledge base attached.

At a minimum, Amazon Bedrock Agents need to be "prepared" for configuration changes to be ready for testing. Optionally, "versions" can be used to track configuration changes and "aliases" map a static identifier to point to an updateable version ID.

In [ ]:
agent_prepare_resp = bedrock_agent_client.prepare_agent(agentId=agent_id)
bedrock_agent_utils.wait_for_agent_prepare(
    agent_id, bedrock_agent_client=bedrock_agent_client
)

In [ ]:
create_alias_resp = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName="KB-ONLY",
    description="Version with only Knowledge Base connected (no hallucination checker tool)",
)
kb_only_alias_id = create_alias_resp["agentAlias"]["agentAliasId"]

bedrock_agent_utils.wait_for_agent_alias(
    agent_id, kb_only_alias_id, bedrock_agent_client=bedrock_agent_client
)

Once the agent is prepared, we can try asking it one of the questions from the dataset:

In [ ]:
q = questions_df.iloc[0].question
print(f"Q: {q}\n--------\nA:")
ans = bedrock_agent_utils.invoke_bedrock_agent(
    agentId=agent_id,
    agentAliasId=kb_only_alias_id,
    sessionId=str(uuid.uuid1()),
    bedrock_agent_runtime_client=bedrock_agent_runtime,
    inputText=q,
)
print(ans)

## Attach the hallucination review tool

To expose our AWS Lambda function as a tool the agent can call when needed, we define an "action group" linked to the function and provide the schema of what parameters it expects to receive.

Note that one Lambda function can actually implement multiple tools or "functions", but in this case we only support the one. The schema here corresponds to the implementation in [infra/hallucination-notifier-lambda/main.py](infra/hallucination-notifier-lambda/main.py):

In [ ]:
create_action_group_resp = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion="DRAFT",
    actionGroupExecutor={"lambda": lambda_function_arn},
    actionGroupName="HallucinationDetectionActionGroup",
    functionSchema={
        "functions": [
            {
                "name": "detect_measure_hallucination",
                "description": "detect potential hallucinations in a knowledge base response, and return an edited final answer",
                "parameters": {
                    "question": {
                        "description": "user question on Amazon Bedrock",
                        "required": True,
                        "type": "string",
                    },
                    "kbResponse": {
                        "description": "knowledge base retrieved response for the user question on Amazon Bedrock",
                        "required": True,
                        "type": "string",
                    },
                },
            }
        ],
    },
    description="Actions for executing hallucination detection and next steps based on the generated answers to the user question",
)

As above, we'll need to "prepare" the draft agent before it's ready to try out.

This time we won't bother to create a version-tracking "alias" though - just use the latest prepared draft:

In [ ]:
agent_prepare_resp = bedrock_agent_client.prepare_agent(agentId=agent_id)
bedrock_agent_utils.wait_for_agent_prepare(
    agent_id, bedrock_agent_client=bedrock_agent_client
)

## Try out the fully-configured agent

Now the agent has been configured with both the knowledge base and the custom hallucination review tool, we can ask it a question likely to trigger the human intervention to see how it responds.

**Note:** Due to the probabilistic and rapidly-evolving nature of today's LLMs, you might need to try a couple of times or edit your question to trigger the custom guardrail.

In [ ]:
q = "Who lives in the town of Bedrock?"
print(f"Q: {q}\n--------\nA:")
ans = bedrock_agent_utils.invoke_bedrock_agent(
    agentId=agent_id,
    sessionId=str(uuid.uuid1()),
    bedrock_agent_runtime_client=bedrock_agent_runtime,
    inputText=q,
)
print(ans)

### Monitoring the SNS messages received for Human in the Loop setup

If you completed the optional subscription of your own email address to the outbound SNS topic, and saw the intervention was triggered above, then you should receive an example email detailing the situation and requesting support.

You can also monitor the `NumberOfMessagesPublished` metric over time for your SNS topic in Amazon CloudWatch as detailed [here in the SNS developer guide](https://docs.aws.amazon.com/sns/latest/dg/sns-monitoring-using-cloudwatch.html) - or try following [this direct link](https://console.aws.amazon.com/cloudwatch/home?#metricsV2?graph=~(view~'timeSeries~stacked~false~)&query=~'*7bAWS*2fSNS*2cTopicName*7d) and selecting your topic's metric from the list.

Alternatively, you can review the CloudWatch Logs of your deployed Lambda function as described [here in the Lambda developer guide](https://docs.aws.amazon.com/lambda/latest/dg/monitoring-cloudwatchlogs-view.html): Find your function in the [AWS Lambda Console](https://console.aws.amazon.com/lambda/home?#/functions) and click on the "Monitor" tab for a direct link to the relevant CloudWatch log group.

## Structured evaluation

Interactive testing is useful... but to enable scalable and iterative improvement of the agent, we need an automated way to run a set of test cases and evaluate the results.

The cell below loops through the full example dataset to test each question and show the results:

In [ ]:
%%time

USER_PROMPT_TEMPLATE = """Question: {question}

Given an input question, you will search the Knowledge Base on Bedrock User Guide to answer the user question. 
If the knowledge base search results does not return any answer you can try answering it to the best of your ability but do not answer anything you do not know. Do not hallucinate.
Using this knowledge base search results you will ALWAYS execute the appropriate action group API to measure and detect the hallucination on that knowledge base search results.

Remove any XML tags from the knowledge base search results and final user response.
"""

agent_answers = list()
for index, row in questions_df.iterrows():
    session_id = str(uuid.uuid1())
    final_agent_answer = None
    question_id = row["question_id"]
    question_text = row["question"]
    gt_answer = row["ground_truth_answer"]
    logger.info(
        f"-------------Question ID :: {question_id} Question_text :: {question_text} -------------------"
    )
    final_agent_answer = bedrock_agent_utils.invoke_bedrock_agent(
        agentId=agent_id,
        sessionId=str(uuid.uuid1()),
        bedrock_agent_runtime_client=bedrock_agent_runtime,
        inputText=USER_PROMPT_TEMPLATE.format(question=question_text),
        # enableTrace = True,
        endSession=False,
    )

    time.sleep(60)
    agent_answers.append(final_agent_answer)
    logger.info(final_agent_answer)

## Challenge Exercise :: Try it Yourself!

<div style="border: 4px solid coral; text-align: left; margin: auto;">
    <br>
    <p style="text-align: center; margin: auto;"><b>Try the following exercises on this lab and note the observations.</b></p>
<p style=" text-align: left; margin: auto;">
<ol>
 <li>Try a new set of questions to test against the agent, reference the Amazon Bedrock User Guide to come up with these questions. </li>
<li> Notice the questions where the human in the loop are getting invoked? Does question reframing/rewriting help avoid it? </li>
<li> Try different chunking strategies supported by Bedrock Knowledge base and ask the same set of questions to compare and contrast against each chunking strategy for this use-case. </li>
<li> Try additional Ragas metrics from the documentation: <a href="https://docs.ragas.io/en/v0.1.21/concepts/metrics/index.html">Ragas metrics</a> </li>
    <li> Try different open source PDF(s) to verify . </li>
</ol>
<br>
</p>
</div>

## Conclusion

In this sample we explored some more advanced hallucination mitigation options, including:

1. How Open Source libraries like [Ragas](https://docs.ragas.io/en/latest/) can be used to build custom hallucination detection guardrails apart from Amazon Bedrock's built-in [Guardrail features](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html)
2. A more-advanced "guardrail-as-a-tool" pattern in which agents can be given autonomy to decide for themselves when to run additional checks on generated answers - versus more rule-based guardrail workflows.

In general, it's often useful to start from fully-managed options like Amazon Bedrock Guardrails first - and enable those deterministically to run on every agent invocation. However, these alternative patterns could be useful in cases where more custom, domain-specific checks are needed or it's shown the agent is reliably able to determine when a guardrail check is relevant - reducing potential guardrail costs.

## Clean-up

Once you're done experimenting, remember to clean up created AWS resources in order to avoid ongoing costs.

First, un-comment and run the code cell below to delete the Bedrock Agent we created from the notebook (you can also review your agents in the [Amazon Bedrock Console](https://console.aws.amazon.com/bedrock/home?#/agents)):

In [ ]:
bedrock_agent_client.delete_agent(agentId=agent_id)

As well as the agent itself, you'll need to separately delete the two CloudFormation infrastructure stacks we deployed - which can be done via the [AWS CloudFormation Console](https://console.aws.amazon.com/cloudformation/home?#/stacks)
1. The example Knowledge Base stack, and
2. The Hallucination detector Lambda stack